In [0]:
# bibliotecas usadas
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score 
from mpl_toolkits.mplot3d import Axes3D, axes3d
%matplotlib inline

In [0]:
# funcao normatiza dados
def Normatiza(x):
    strings=list(x)
    for i in strings:
        max_x=x[i].max()
        min_x=x[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=(x[i]-a)/b
    return x

In [0]:
# funcao retorna os dados a forma original
def Original(x,x_old):
    strings=list(x) # todas as colunas
    for i in strings:
        max_x=x_old[i].max()
        min_x=x_old[i].min()
        a=(max_x+min_x)/2
        b=(max_x-min_x)/2
        x[i]=x[i]*b+a
    return x

In [49]:
!git clone http://github.com/ucfilho/ANN
%cd ANN

fatal: destination path 'ANN' already exists and is not an empty directory.
/content/ANN/ANN/ANN


In [50]:
DAT=pd.read_csv("Polimero_maio_21_2015.txt",sep='\t')
print(DAT.head())

   Time     TC     Y  MI    MW
0  80.0  170.0  76.5  62  2940
1  80.0  180.0  77.0  60  3470
2  90.0  170.0  78.0  66  3680
3  90.0  180.0  79.5  59  3890
4  85.0  175.0  79.9  72  3480


In [0]:
# separa as colunas de interesse
Index=range(5)
Dat_Select=[]
Dat_Select=DAT.iloc[:,Index].copy()

In [0]:
# separa os dados em normatizados e originais
DAT_OLD=[]
DAT_OLD=Dat_Select.copy()
DAT=Normatiza(Dat_Select)

In [0]:
# separa os dados em treino e teste
train=DAT.sample(frac=1.0,random_state=None) # poderia ser frac=0.5? sim ou nao?
test=DAT.drop(train.index)

In [0]:
# obtem as variaveis independentes (treino e teste)
x_train=train.iloc[:,[0,1]]
x_test=test.iloc[:,[0,1]]
X_OLD=DAT_OLD.iloc[:,[0,1]]

In [55]:
# obtem as variaveis dependentes (treino e teste)
Index=[2,3,4]
y_train=train.iloc[:,Index]
y_test=test.iloc[:,Index]
Y_OLD=DAT_OLD.iloc[:,Index]
print(Y_OLD)
print(x_train)

       Y  MI    MW
0   76.5  62  2940
1   77.0  60  3470
2   78.0  66  3680
3   79.5  59  3890
4   79.9  72  3480
5   80.3  69  3200
6   80.0  68  3410
7   79.7  70  3290
8   79.8  71  3500
9   78.4  68  3360
10  75.6  71  3020
11  78.5  58  3630
12  77.0  57  3150
        Time        TC
9   1.000000  0.000000
11  0.000000  1.000000
0  -0.707214 -0.707214
6   0.000000  0.000000
2   0.707214 -0.707214
8   0.000000  0.000000
10 -1.000000  0.000000
1  -0.707214  0.707214
5   0.000000  0.000000
7   0.000000  0.000000
12  0.000000 -1.000000
4   0.000000  0.000000
3   0.707214  0.707214


In [0]:
# define a rede neural 
clf = MLPRegressor(solver='lbfgs',activation='tanh',alpha=1e-5,hidden_layer_sizes=(6), 
                    random_state=1)

In [57]:
# treina a rede neural
clf.fit(x_train, y_train)

MLPRegressor(activation='tanh', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=6, learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [58]:
# usa a rede neural treina em calculos para treino e teste
y_calc_train=clf.predict(x_train)
x_test=x_train
y_calc_test=clf.predict(x_test)
print(y_calc_train)
print(Y_OLD)

[[ 0.19162437  0.46890113 -0.11523183]
 [ 0.23089721 -0.85116368  0.46348445]
 [-0.61728296 -0.33259146 -1.00179793]
 [ 0.8469871   0.7319892  -0.08280818]
 [ 0.0222893   0.20036214  0.55806667]
 [ 0.8469871   0.7319892  -0.08280818]
 [-1.00033998  0.87380151 -0.82571793]
 [-0.40240044 -0.61595166  0.10374218]
 [ 0.8469871   0.7319892  -0.08280818]
 [ 0.8469871   0.7319892  -0.08280818]
 [-0.40501671 -0.99969978 -0.55823215]
 [ 0.8469871   0.7319892  -0.08280818]
 [ 0.66026561 -0.73510632  0.99757136]]
       Y  MI    MW
0   76.5  62  2940
1   77.0  60  3470
2   78.0  66  3680
3   79.5  59  3890
4   79.9  72  3480
5   80.3  69  3200
6   80.0  68  3410
7   79.7  70  3290
8   79.8  71  3500
9   78.4  68  3360
10  75.6  71  3020
11  78.5  58  3630
12  77.0  57  3150


In [0]:
# transforma a saida da rede em dataframe 
y_calc_train=pd.DataFrame(y_calc_train)
y_calc_test=pd.DataFrame(y_calc_test)
col_names=list(y_train)
y_calc_train.columns = col_names
y_calc_test.columns = col_names

In [60]:
# transforma dos dados calculados na forma original (nao normatizado)
y_calc_train=Original(y_calc_train,Y_OLD)
y_calc_test=Original(y_calc_test,Y_OLD)
print(y_calc_train)
print(Y_OLD)

            Y         MI           MW
0   78.400317  68.016758  3360.264880
1   78.492608  58.116272  3635.155115
2   76.499385  62.005564  2939.145984
3   79.940420  69.989919  3375.666112
4   78.002380  66.002716  3680.081668
5   79.940420  69.989919  3375.666112
6   75.599201  71.053511  3022.783985
7   77.004359  59.880363  3464.277533
8   79.940420  69.989919  3375.666112
9   79.940420  69.989919  3375.666112
10  76.998211  57.002252  3149.839730
11  79.940420  69.989919  3375.666112
12  79.501624  58.986703  3888.846394
       Y  MI    MW
0   76.5  62  2940
1   77.0  60  3470
2   78.0  66  3680
3   79.5  59  3890
4   79.9  72  3480
5   80.3  69  3200
6   80.0  68  3410
7   79.7  70  3290
8   79.8  71  3500
9   78.4  68  3360
10  75.6  71  3020
11  78.5  58  3630
12  77.0  57  3150


In [0]:
def MW_calc(x_test,Y_OLD,clf):
    y_calc_test=clf.predict(x_test)
    y_calc_test=Original(y_calc_test,Y_OLD)
    return y_calc_test['MW']
def MI_calc(x_test,Y_OLD,clf):
    y_calc_test=clf.predict(x_test)
    y_calc_test=Original(y_calc_test,Y_OLD)
    return y_calc_test['MI']
def Y_calc(x_test,Y_OLD,clf):
    y_calc_test=clf.predict(x_test)
    y_calc_test=Original(y_calc_test,Y_OLD)
    return y_calc_test['Y']

def Desirability(y,U,T,L):
    if y<L:
        d=0
    if (y>=L)&(y<=T):
        d=(y-L)/(T-L)
    if (y>=T)&(y<+U):
        d=(U-y)/(U-T)
    if (y>U):
        d=0
    return d

In [0]:
def function(x):
    Time=x[0]
    TC=x[1]
    MW=MW_calc(Time, TC)
    MI=MI_calc(Time, TC)
    Y=Y_calc(Time, TC)
    U=3890
    T=3400
    L=2940
    d1=Desirability(MW,U,T,L)
    U=68
    T=65
    L=62
    d2=Desirability(MI,U,T,L)
    U=80
    T=77.5
    L=75.6
    d3=Desirability(Y,U,T,L)
    fun = (-1)*(d1*d2*d3)**(1/3)
    return fun

In [63]:
x = np.linspace(-6, 6, 5)
y = np.linspace(-6, 6, 5)

X, Y = np.meshgrid(x, y)
print(X)
print(Y)

[[-6. -3.  0.  3.  6.]
 [-6. -3.  0.  3.  6.]
 [-6. -3.  0.  3.  6.]
 [-6. -3.  0.  3.  6.]
 [-6. -3.  0.  3.  6.]]
[[-6. -6. -6. -6. -6.]
 [-3. -3. -3. -3. -3.]
 [ 0.  0.  0.  0.  0.]
 [ 3.  3.  3.  3.  3.]
 [ 6.  6.  6.  6.  6.]]


In [64]:
print("hello world,I am evolving!!!")

hello world,I am evolving!!!


*PS*: no planejamento de experimentos utilizando o Statistica -e-papers
Y=80
MI=65,044
MW=3586,1

tempo=86,65 e Temperatura=178,77